In [4]:
import torch
from torchtext.legacy import data
import pandas as pd
import os
import torch.nn as nn
import time
from utils import count_parameters
import torch.optim as optim
import random


SEED= 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cpu')

### Load Data

In [6]:
from load_data import load_dataset
TEXT, LABEL, vocab_size,  train_iter, valid_iter, test_iter = load_dataset()

Unique tokens in TEXT vocabulary: 9990
Unique tokens in LABEL vocabulary: 3
[('the', 3882), (',', 3215), ('.', 3049), ('of', 2207), ('in', 2051), ('to', 1944), ('and', 1759), ('-', 1314), ('a', 1098), ('for', 896), ("'s", 831), ('eur', 653), ('is', 601), ('will', 565), ('company', 557), ('from', 528), ('on', 527), ('as', 483), ('with', 449), ('its', 429)]


In [7]:
print('Training Size: %d'%len(train_iter.dataset.examples))
print('Validation Size: %d'%len(valid_iter.dataset.examples))
print('Test Size: %d'%len(test_iter.dataset.examples))

Training Size: 4146
Validation Size: 1037
Test Size: 1296


In [8]:
train_label_count_dict = {'positive':0, 'negative':0, 'neutral':0}
for lab in train_iter.dataset.examples:
    train_label_count_dict[lab.label]+=1
print('Training Label count:')
print(train_label_count_dict)

print('###############')
val_label_count_dict = {'positive':0, 'negative':0, 'neutral':0}
for lab in valid_iter.dataset.examples:
    val_label_count_dict[lab.label]+=1
print('Validation Label count:')
print(val_label_count_dict)

print('###############')
test_label_count_dict = {'positive':0, 'negative':0, 'neutral':0}
for lab in test_iter.dataset.examples:
    test_label_count_dict[lab.label]+=1
print('Test Label count:')
print(test_label_count_dict)

Training Label count:
{'positive': 1321, 'negative': 688, 'neutral': 2137}
###############
Validation Label count:
{'positive': 317, 'negative': 162, 'neutral': 558}
###############
Test Label count:
{'positive': 378, 'negative': 205, 'neutral': 713}


### RNN

In [9]:
from RNN import RNN
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 3

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model.to(device)
criterion = nn.CrossEntropyLoss(reduction='mean').to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,091,419 trainable parameters


##### Training RNN

In [10]:
for batch in train_iter:
    print(batch)
    break


[torchtext.legacy.data.batch.Batch of size 64]
	[.sentence]:[torch.LongTensor of size 17x64]
	[.label]:[torch.LongTensor of size 64]


In [31]:
batch.dataset.examples[0].label

'positive'

In [ ]:
from utils import train, evaluate, macro_f1
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_macro_f1 = train(model, train_iter, optimizer, criterion, macro_f1)
    valid_loss, valid_macro_f1 = evaluate(model, valid_iter, criterion, macro_f1)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_macro_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_macro_f1*100:.2f}%')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\musta\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-0e70ecdad598>", line 10, in <module>
    train_loss, train_macro_f1 = train(model, train_iter, optimizer, criterion, macro_f1)
  File "C:\Users\musta\Github\FinalProject\LSTM_Model\utils.py", line 39, in train
    predictions = model(batch.sentence).squeeze(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\musta\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent ca

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\musta\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-0e70ecdad598>", line 10, in <module>
    train_loss, train_macro_f1 = train(model, train_iter, optimizer, criterion, macro_f1)
  File "C:\Users\musta\Github\FinalProject\LSTM_Model\utils.py", line 39, in train
    predictions = model(batch.sentence).squeeze(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\musta\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent ca

In [8]:
for i in range(35000):
    a = 1+1